# Setup & Data loading

In [2]:
# Setup & Data Loading
import json
import re
import html
from pathlib import Path
from typing import Any, Dict, List, Tuple

import pandas as pd
from bs4 import BeautifulSoup


def load_data(file_path: str | Path) -> pd.DataFrame:
    """loads the raw dataset into a DataFrame or list.
    Supports JSON array or JSONL inputs.
    """
    p = Path(file_path)
    with p.open('r', encoding='utf-8') as f:
        first = f.read(1)
        f.seek(0)
        if first == '[':
            data = json.load(f)
        else:
            data = [json.loads(line) for line in f if line.strip()]
    return pd.DataFrame(data)


def inspect_data(df: pd.DataFrame) -> None:
    """shows basic info and samples to understand structure."""
    print('Shape:', df.shape)
    print('Columns:', list(df.columns))
    display(df.head(3))
    display(df.sample(min(3, len(df))))


# Text Cleaning and Normalization

In [3]:
def clean_text(text: str) -> str:
    """Cleans and normalizes raw text fields.
    Steps:
    - Unescape HTML entities (&amp;, &lt;, etc.)
    - Remove HTML tags
    - Lowercase all text
    - Remove URLs, punctuation, digits, and special symbols
    - Collapse multiple spaces
    Returns a clean text string.
    """
    if not isinstance(text, str) or not text.strip():
        return ""

    # Unescape HTML and strip tags
    text = html.unescape(text)
    text = BeautifulSoup(text, "html.parser").get_text(separator=" ")

    # Lowercase
    text = text.lower()

    # Remove URLs
    text = re.sub(r"http\S+|www\S+", " ", text)

    # Remove punctuation, digits, and non-alphabetic characters
    text = re.sub(r"[^a-z\s]", " ", text)

    # Collapse multiple spaces
    text = re.sub(r"\s+", " ", text).strip()

    return text

# Tokenization & Text Processing

In [4]:
# Part 3 – Tokenization & Text Processing
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize

# Make sure NLTK resources are available (run once)
nltk.download('punkt')
nltk.download('stopwords')

STOP_WORDS = set(stopwords.words('english'))
STEMMER = SnowballStemmer('english')


def tokenize_text(text: str) -> List[str]:
    """Splits text into individual tokens (words)."""
    if not isinstance(text, str) or not text.strip():
        return []
    return word_tokenize(text)


def remove_stopwords(tokens: List[str]) -> List[str]:
    """Removes common stopwords from token list."""
    return [t for t in tokens if t not in STOP_WORDS]


def stem_tokens(tokens: List[str]) -> List[str]:
    """Applies stemming to reduce tokens to their root form."""
    return [STEMMER.stem(t) for t in tokens]


def preprocess_text(text: str) -> List[str]:
    """Applies full text preprocessing pipeline:
    - Clean text (using clean_text)
    - Tokenize
    - Remove stopwords
    - Stem tokens
    Returns a list of processed tokens.
    """
    cleaned = clean_text(text)
    tokens = tokenize_text(cleaned)
    tokens = remove_stopwords(tokens)
    tokens = stem_tokens(tokens)
    return tokens


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Record-Level Preprocessing

In [5]:
# Record-Level Preprocessing

def preprocess_product_details(details: Any) -> str:
    """flattens and cleans the list of product attributes."""
    if isinstance(details, list):
        parts = []
        for item in details:
            if isinstance(item, dict):
                for k, v in item.items():
                    parts.append(f"{k}: {v}")
            else:
                parts.append(str(item))
        return clean_text(' '.join(parts))
    return clean_text(str(details))


def normalize_numeric_fields(record: Dict[str, Any]) -> Dict[str, Any]:
    """converts numeric strings (price, rating, etc.) to numbers."""
    rec = dict(record)
    for key in ('price', 'rating', 'num_reviews', 'discount', 'old_price'):
        if key in rec and rec[key] is not None:
            val = rec[key]
            if isinstance(val, (int, float)):
                continue
            s = str(val)
            s = s.replace(',', '.').replace('€', '').replace('$', '').strip()
            s = re.sub(r"[^0-9.\-]", "", s)
            try:
                rec[key] = float(s) if ('.' in s or '-' in s) else int(s)
            except Exception:
                rec[key] = None
    return rec


def preprocess_record(record: Dict[str, Any]) -> Dict[str, Any]:
    """applies all cleaning to one product entry."""
    rec = normalize_numeric_fields(record)
    title = clean_text(rec.get('title', ''))
    description = clean_text(rec.get('description', ''))
    category = clean_text(rec.get('category', ''))
    subcategory = clean_text(rec.get('subcategory', ''))
    details = preprocess_product_details(rec.get('details') or rec.get('product_details') or [])

    full_text = ' '.join([title, description, category, subcategory, details]).strip()
    tokens = preprocess_text(full_text)

    rec['title_clean'] = title
    rec['description_clean'] = description
    rec['category_clean'] = category
    rec['subcategory_clean'] = subcategory
    rec['details_clean'] = details
    rec['tokens'] = tokens
    return rec


def preprocess_corpus(df: pd.DataFrame) -> pd.DataFrame:
    """applies preprocess_record to the whole dataset."""
    return df.apply(lambda row: pd.Series(preprocess_record(row.to_dict())), axis=1)


# Export & Summary

In [6]:
# Export & Summary

def save_processed_data(df: pd.DataFrame, path: str | Path) -> None:
    """saves the cleaned dataset to JSON or CSV."""
    p = Path(path)
    p.parent.mkdir(parents=True, exist_ok=True)
    if p.suffix.lower() == '.csv':
        df.to_csv(p, index=False)
    else:
        df.to_json(p, orient='records', force_ascii=False)


def summarize_preprocessing(df: pd.DataFrame) -> None:
    """prints basic stats or sample processed data."""
    print('Rows:', len(df))
    print('Columns:', len(df.columns))
    print('Sample tokens:')
    display(df[['title_clean', 'tokens']].head(5))


# Run Processing Pipeline

In [7]:
# 1. Load the raw dataset
df_raw = load_data("../../data/fashion_products_dataset.json")

# 2. Inspect a few entries
inspect_data(df_raw)

# 3. Preprocess the corpus
df_processed = preprocess_corpus(df_raw)

# 4. Summarize to verify everything looks fine
summarize_preprocessing(df_processed)

# 5. Save the processed dataset for future use
save_processed_data(df_processed, "data/processed_corpus.json")

Shape: (28080, 17)
Columns: ['_id', 'actual_price', 'average_rating', 'brand', 'category', 'crawled_at', 'description', 'discount', 'images', 'out_of_stock', 'pid', 'product_details', 'seller', 'selling_price', 'sub_category', 'title', 'url']


,_id,actual_price,average_rating,brand,category,crawled_at,description,discount,images,out_of_stock,pid,product_details,seller,selling_price,sub_category,title,url
0,fa8e22d6-c0b6-5229-bb9e-ad52eda39a0a,"2,999",3.9,York,Clothing and Accessories,1612987911000,Yorker trackpants made from 100% rich combed c...,69% off,[https://rukminim1.flixcart.com/image/128/128/...,False,TKPFCZ9EA7H5FYZH,"[{'Style Code': '1005COMBO2'}, {'Closure': 'El...",Shyam Enterprises,921,Bottomwear,Solid Women Multicolor Track Pants,https://www.flipkart.com/yorker-solid-men-mult...
1,893e6980-f2a0-531f-b056-34dd63fe912c,"1,499",3.9,York,Clothing and Accessories,1612987912000,Yorker trackpants made from 100% rich combed c...,66% off,[https://rukminim1.flixcart.com/image/128/128/...,False,TKPFCZ9EJZV2UVRZ,"[{'Style Code': '1005BLUE'}, {'Closure': 'Draw...",Shyam Enterprises,499,Bottomwear,Solid Men Blue Track Pants,https://www.flipkart.com/yorker-solid-men-blue...
2,eb4c8eab-8206-59d0-bcd1-a724d96bf74f,"2,999",3.9,York,Clothing and Accessories,1612987912000,Yorker trackpants made from 100% rich combed c...,68% off,[https://rukminim1.flixcart.com/image/128/128/...,False,TKPFCZ9EHFCY5Z4Y,"[{'Style Code': '1005COMBO4'}, {'Closure': 'El...",Shyam Enterprises,931,Bottomwear,Solid Men Multicolor Track Pants,https://www.flipkart.com/yorker-solid-men-mult...


,_id,actual_price,average_rating,brand,category,crawled_at,description,discount,images,out_of_stock,pid,product_details,seller,selling_price,sub_category,title,url
5111,6c2e19d7-4e46-5dc3-9de1-18dad91044dd,"2,100",,FASHION WOU,Clothing and Accessories,1612991376000,Rope offers this pure maroon trendy kurta whic...,57% off,[https://rukminim1.flixcart.com/image/128/128/...,False,KTAFYMH86WNDCN3G,"[{'Ideal For': 'Men'}, {'Length Type': 'Short'...",SHASHI TRADERS(New Sell,895,"Kurtas, Ethnic Sets and Bottoms",Men Solid Cotton Blend Straight Kurta (Multic...,https://www.flipkart.com/fashion-would-men-sol...
22434,e8454c8a-066c-5328-9594-23071350af0e,"2,499",3.3,Purple Sta,Clothing and Accessories,1613003520000,"stylish shirt for men, FORMAL SHIRTr Design sh...",77% off,[https://rukminim1.flixcart.com/image/128/128/...,False,SHTFEBYPJUXY6KHG,"[{'Pack of': '1'}, {'Style Code': 'PS-85-XL'},...",Purplestate,561,Topwear,Men Slim Fit Solid Slim Collar Casual Shirt,https://www.flipkart.com/purple-state-men-soli...
25146,fe20eabc-c5aa-5d2c-9d68-26071c8b06ff,699,4,TOM BU,Clothing and Accessories,1613005273000,,70% off,[https://rukminim1.flixcart.com/image/128/128/...,False,TSHFRCDKXMXF3HVZ,"[{'Type': 'Round Neck'}, {'Sleeve': 'Half Slee...",TOMBURG,207,Topwear,Self Design Men Round Neck Blue T-Shirt,https://www.flipkart.com/tom-burg-self-design-...


C:\Users\Usuario\AppData\Local\Temp\ipykernel_9992\3436818360.py:16: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, "html.parser").get_text(separator=" ")


Rows: 28080
Columns: 23
Sample tokens:


,title_clean,tokens
0,solid women multicolor track pants,"[solid, women, multicolor, track, pant, yorker..."
1,solid men blue track pants,"[solid, men, blue, track, pant, yorker, trackp..."
2,solid men multicolor track pants,"[solid, men, multicolor, track, pant, yorker, ..."
3,solid women multicolor track pants,"[solid, women, multicolor, track, pant, yorker..."
4,solid women brown grey track pants,"[solid, women, brown, grey, track, pant, yorke..."
